<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
        Get Nim Endpoints
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

# Import required packages

In [10]:
import yaml
import os
import json
import pandas as pd
from teradataml.sdk import ClientCredentialsAuth
from teradataml.sdk.modelops import ModelOpsClient, Projects, TrainedModels, Deployments, Jobs
from teradataml import DataFrame, create_context
from getpass import getpass

In [3]:
def list_to_frame(d, columns):
    df = pd.DataFrame([[l.get(column) for column in columns] for l in d], columns=columns)
    return df

# Create Client object

In [ ]:
# Create ClientCredentialsAuth object to store client creds required to authenticate to ModelOps.
cc_obj = ClientCredentialsAuth(auth_client_id=getpass(prompt="auth_client_id"),
                               auth_client_secret=getpass(prompt="auth_client_secret"),
                               auth_token_url=getpass(prompt="auth_token_url"))

In [9]:
# Authenticate to Model Ops Rest Framework.
client = ModelOpsClient(base_url=getpass(prompt=base_url), auth=cc_obj, ssl_verify=False)
client

Certificate validation disabled. Adding certificate verification is strongly advised


In [ ]:
context=create_context(host=hostname, username=username, password=password)

In [7]:
p = Projects(client=client)
tr = TrainedModels(client=client)
dep = Deployments(client=client)
jbs = Jobs(client=client)

# Required models.

In [24]:
# Needed models
embed_model_external_id = "llama-3.2-nv-embedqa-1b-v2"
chat_completions_model_external_id = "mistral-7b-instruct-v0.3"

# Steps to get URL endpoint when model is already deployed.

## Get Projects list in teradataml DataFrame.

In [12]:
from teradataml import DataFrame
all_projects_dict = p.find_all(return_dict=True)["_embedded"]["projects"]
DataFrame.from_records(data=all_projects_dict, columns=["id", "name", "description", "group_id", "archived"])

id,name,description,group_id,archived
db00b631-3a79-42bc-bb22-9b2dc662a83f,Modelops test,Modelops test,None,0
e4a9e16e-d2ac-4d3a-b4b9-c03001aaab84,st255077,st255077 personal project,None,0
70d4659b-92a2-4723-841a-9ba5629b5f27,Demo,Example ML Use Cases,None,0
92e50b14-4572-4d2a-870c-7dcb4b86c704,test new,test new,None,0
14901492-5081-4a11-8445-f42aa142ae68,new project for user test user,new project for user test user,None,1
555fc04a-8905-45a1-b643-5e192024ccb3,test project 1,test project 1,None,0
631fffcf-8cca-49a5-ba29-33dfeb69bca2,gc255040,test project,None,1
39b0a327-7c40-4cb9-b4a4-39bc84925111,Personal_git_project,Personal git project,None,0
92bd7e23-93d5-4147-a82c-1ed2668c5d94,test project,Test project (updated),None,0
21305e37-8822-4008-ad9f-381222a66350,sit_test_user_1_project,SA255083,None,0


# Use "NIM" project

In [14]:
project_id = "5b522a3a-9015-405f-b57d-b2e48ca877c4"

In [15]:
client.set_project_id(project_id=project_id)

# List all active deployments.

In [ ]:
current_deployments = dep.find_active(projection="expandDeployment", return_dict=True)["_embedded"]["deployments"]

In [26]:
deployed_list = []
for md in current_deployments:
    model_detail = {}
    model_detail["id"] = md["trainedModel"]["id"]
    model_detail["externalId"] = md["trainedModel"]["externalId"]
    model_detail["status"] = md["status"]
    model_detail["ownerId"] = md["ownerId"]
    model_detail["projectId"] = md["projectId"]
    model_detail["baseUrl"] = md["metadata"]["baseUrl"]
    deployed_list.append(model_detail)
pd.DataFrame(deployed_list, columns=["id", "externalId", "status", "ownerId", "projectId", "baseUrl"])

,id,externalId,status,ownerId,projectId,baseUrl
0,96bd9549-be12-4fdc-bbbd-dbef8a169be0,llama-3.2-nv-embedqa-1b-v2,DEPLOYED,service-account-modelops-cli,5b522a3a-9015-405f-b57d-b2e48ca877c4,https://gpudev1-modelops.labs.teradata.com/ser...
1,1107c851-e108-449f-bb6b-b7183ce9ffef,mistral-7b-instruct-v0.3,DEPLOYED,service-account-modelops-cli,5b522a3a-9015-405f-b57d-b2e48ca877c4,https://gpudev1-modelops.labs.teradata.com/ser...
2,f4900e57-ba7d-420c-ab9a-411964fbb99f,llama-3.1-8b-base,DEPLOYED,kmsadmin,5b522a3a-9015-405f-b57d-b2e48ca877c4,https://gpudev1-modelops.labs.teradata.com/ser...
3,c6900932-d030-4387-ac7f-dfec1c04acb4,nemotron-4-340b-reward,DEPLOYED,admin,5b522a3a-9015-405f-b57d-b2e48ca877c4,https://gpudev1-modelops.labs.teradata.com/ser...


# Get the baseUrl for the embeddings_model: llama-3.2-nv-embedqa-1b-v2 from the current active deployements.

In [16]:
current_deployments = dep.find_active(projection="expandDeployment", return_dict=True)["_embedded"]["deployments"]
base_url = next((dep["metadata"]["baseUrl"] for dep in current_deployments if dep["trainedModel"]['externalId'] == embed_model_external_id), None)
print(base_url)

https://gpudev1-modelops.labs.teradata.com/serving/96bd9549-be12-4fdc-bbbd-dbef8a169be0/v1


In [25]:
base_url = next((dep["metadata"]["baseUrl"] for dep in current_deployments if dep["trainedModel"]['externalId'] == chat_completions_model_external_id), None)
print(base_url)

https://gpudev1-modelops.labs.teradata.com/serving/1107c851-e108-449f-bb6b-b7183ce9ffef/v1


## Required models not found and need to be deployed.
### 1. Get the list of trained models
### 2. Get the model ids
### 3. deploy the model ( We can retire any model if needed)

### 1. Get the list of trained models

In [23]:
all_trained_models = tr.find_all(return_dict=True, size=1000)["_embedded"]["trainedModels"]
all_trained_models = list_to_frame(all_trained_models, columns=["id", "externalId", "archived", "metadata"])
all_trained_models

                                      id  \
0   0de99f4f-509f-42cf-9251-e3c1dbf45dda   
1   4bdef745-c753-4920-99c2-08634dad58b1   
2   7da75adc-c497-4b17-9898-0059d8733d0f   
3   650b7e21-ee4a-4235-9b93-bc50870cd444   
4   9827c8a5-2f08-4bd3-8d64-fe6d217a4095   
5   1107c851-e108-449f-bb6b-b7183ce9ffef   
6   c03cc65f-583e-4d27-ae62-f6304bf30e33   
7   feed36cf-1efb-44aa-b341-652d1b696a51   
8   972b8fea-0f8c-4f95-8e11-4555f626996f   
9   009d4526-f22f-4e3b-9d8a-c3ab78f0ca89   
10  f1c01574-bc8a-4836-ab44-d03af8102a4d   
11  45de5788-59b6-4faf-a10b-007521318cc9   
12  e0d321c8-8cd9-415f-a619-42944cb9fcca   
13  0176054d-975f-4087-992b-c7c00d11b7b3   
14  011f3477-5785-4140-842d-6cca82177fb6   
15  447be798-9822-4476-9ab9-348f0a03a864   
16  012d04d3-b58e-4357-ad58-82042b13c918   
17  8f61a030-7900-40f2-b79e-033dfb905e1f   
18  7490af4f-b4ff-4f51-86c1-1c49cfc20c84   
19  bb0a099a-5400-43c5-acbd-c1b2463a8897   
20  b0da212b-baab-4729-8fd8-6215172da9f7   
21  1234db31-658b-446f-baf3-b534

### Get the id for embeddings model 'llama-3.2-nv-embedqa-1b-v2'.

In [25]:
model_data = all_trained_models[all_trained_models['externalId'].str.fullmatch(embed_model_external_id, na=False)]

In [26]:
model_data

                                      id                  externalId  \
34  96bd9549-be12-4fdc-bbbd-dbef8a169be0  llama-3.2-nv-embedqa-1b-v2   

    archived                                           metadata  
34     False  {'configuration': {'displayName': 'NVIDIA Retr...  


In [27]:
req_model_id = model_data["id"].values[0]

In [28]:
req_model_id

'96bd9549-be12-4fdc-bbbd-dbef8a169be0'

### Deploy the model.

In [ ]:
#Helper function
def deployment_request(gpu=1):
    return {
        "engineType": "LMZ_RESTFUL",
        "engineTypeConfig":
        {
            "dockerImage": "use_an_image_from_model_configuration",
            "engine": "empty",
            "extraConfig":
            { # Any custom configuration that we need in lmz_deployer.sh goes here
                "doStuff": "yes"
            },
            "replicas": 1,
            "resources":
            {
                "gpu": gpu
            }
        },
        "language": "LMZ",
        "publishOnly": "false",
        "customProperties":
        {}
    }

In [ ]:
deploy_job = tr.deploy(id=req_model_id, body=deployment_request(gpu=1), return_dict=True)

In [ ]:
print(f"Client is waiting for job - {deploy_job['id']} to complete......")
wait_for_job_completion(deploy_job['id'])
time.sleep(30)

# Get the base URL for the deployed model

In [ ]:
current_deployments = dep.find_active(projection="expandDeployment", return_dict=True)["_embedded"]["deployments"]
base_url = next((dep["metadata"]["baseUrl"] for dep in current_deployments if dep["trainedModel"]['externalId'] == embed_model_external_id
                ), None)
print(base_url)